In [1]:
# organize imports
import cv2
import imutils
import numpy as np
from pymouse import PyMouse
from sklearn.metrics import pairwise
import dlib
from imutils import face_utils
from scipy.spatial import distance as dist
# global variables
bg = None

In [2]:
#-------------------------------------------------------------------------------
# Function - To find the running average over the background
#-------------------------------------------------------------------------------
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

In [3]:
#-------------------------------------------------------------------------------
# Function - To segment the region of hand in the image
#-------------------------------------------------------------------------------
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff,
                                threshold,
                                255,
                                cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (_, cnts, _) = cv2.findContours(thresholded.copy(),
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)


In [4]:
#-------------------------------------------------------------------------------
# Function - To count the number of fingers in the segmented hand region
#-------------------------------------------------------------------------------
def count(thresholded, segmented):
    # find the convex hull of the segmented hand region
    chull = cv2.convexHull(segmented)

    # find the most extreme points in the convex hull
    extreme_top    = tuple(chull[chull[:, :, 1].argmin()][0])
    extreme_bottom = tuple(chull[chull[:, :, 1].argmax()][0])
    extreme_left   = tuple(chull[chull[:, :, 0].argmin()][0])
    extreme_right  = tuple(chull[chull[:, :, 0].argmax()][0])

    # find the center of the palm
    cX = (extreme_left[0] + extreme_right[0]) // 2
    cY = (extreme_top[1] + extreme_bottom[1]) // 2

    # find the maximum euclidean distance between the center of the palm
    # and the most extreme points of the convex hull
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[extreme_left, extreme_right, extreme_top, extreme_bottom])[0]
    maximum_distance = distance[distance.argmax()]

    # calculate the radius of the circle with 80% of the max euclidean distance obtained
    radius = int(0.7 * maximum_distance)
    
    # find the circumference of the circle
    circumference = (2 * np.pi * radius)

    # take out the circular region of interest which has 
    # the palm and the fingers
    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
    
    # draw the circular ROI
    cv2.circle(circular_roi, (cX, cY), radius, 255, 1)
    
    # take bit-wise AND between thresholded hand using the circular ROI as the mask
    # which gives the cuts obtained using mask on the thresholded hand image
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

    # compute the contours in the circular ROI
    (_, cnts, _) = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # initalize the finger count
    count = 0

    # loop through the contours found
    for c in cnts:
        # compute the bounding box of the contour
        (x, y, w, h) = cv2.boundingRect(c)

        # increment the count of fingers only if -
        # 1. The contour region is not the wrist (bottom area)
        # 2. The number of points along the contour does not exceed
        #     25% of the circumference of the circular ROI
        if ((cY + (cY * 0.25)) > (y + h)) and ((circumference * 0.25) > c.shape[0]):
            count += 1

    return count

In [5]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
 
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
 
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
 
    # return the eye aspect ratio
    return ear

In [27]:
#Threshold for Eye Aspect Ratio
earThreshold=0.15
    
#Threshold for blink duration
clickThreshold=3
    
# initialize accumulated weight
accumWeight = 0.5

#mouse Speed
speed=20
    
#move threshold
moveThreshold=2

In [26]:
#-------------------------------------------------------------------------------
# Main function
#-------------------------------------------------------------------------------
if __name__ == "__main__":
    #PyMouse object
    m=PyMouse()
    
    # For wink detection
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") #use link to get file https://drive.google.com/open?id=1r84y_rYPyGuGiuJhXTjtPQHCpCj6w_Ct 
    
    # grab the indexes of the facial landmarks for the left and
    # right eye, respectively
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    
    
    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0

    # calibration indicator
    calibrated = False

    #count for blink frames
    bCount=0
    
    
    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width=700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our weighted average model gets calibrated
        if num_frames < 30:
            run_avg(gray, accumWeight)
            if num_frames == 1:
                print ("[STATUS] please wait! calibrating...")
            elif num_frames == 29:
                print ("[STATUS] calibration successfull...")       
        else:
            # segment the hand region
            hand = segment(gray)
            
            # detect faces in each frame
            rects = detector(clone, 0)
            
            # loop over the face detections
            for rect in rects:
            # determine the facial landmarks for the face region, then
            # convert the facial landmark (x, y)-coordinates to a NumPy
            # array
                shape = predictor(clone, rect)
                shape = face_utils.shape_to_np(shape)
 
                # extract the left and right eye coordinates, then use the
                # coordinates to compute the eye aspect ratio for both eyes
                leftEye = shape[lStart:lEnd]
                rightEye = shape[rStart:rEnd]
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)
                
                # compute the convex hull for the left and right eye, then
                # visualize each of the eyes
                leftEyeHull = cv2.convexHull(leftEye)
                rightEyeHull = cv2.convexHull(rightEye)
                cv2.drawContours(clone, [leftEyeHull], -1, (0, 255, 0), 1)
                cv2.drawContours(clone, [rightEyeHull], -1, (0, 255, 0), 1)
                
                if (leftEAR+rightEAR)/2>=earThreshold:
                    bCount=0
                else:
                    bCount+=1
                
                
                cv2.putText(clone, "Wink="+str(bCount), (70, 75), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                #cv2.putText(clone, "BlinkRight="+str(bCountR), (70, 105), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                #cv2.putText(clone, "E.A.R Right="+str(rightEAR), (70, 135), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                #cv2.putText(clone, "E.A.R Left="+str(leftEAR), (70, 165), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                    
                    
                if bCount>clickThreshold:          #action for left blink**** assign function to do
                    m.click(m.position()[0],m.position()[1],1)
                    bCount=0
                
                
 

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))

                # count the number of fingers
                fingers = count(thresholded, segmented)
                
                if fingers==1:  #right
                    m.move(m.position()[0]+speed,m.position()[1])
                    cv2.putText(clone, "Going->Right", (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                
                elif fingers==2:   #left
                    m.move(m.position()[0]-speed,m.position()[1])
                    cv2.putText(clone, "Going->Left", (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                
                elif fingers==3:   #up
                    m.move(m.position()[0],m.position()[1]-speed)
                    cv2.putText(clone, "Going->Up"+str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                
                elif fingers==4:   #down
                    m.move(m.position()[0],m.position()[1]+speed)
                    cv2.putText(clone, "Going->Down"+str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                
                
                else:
                    cv2.putText(clone, "Going->Nowhere", (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                
               
                # show the thresholded image
                cv2.imshow("Thesholded", thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break
            
        #reset background
        if keypress == ord("r"):
            num_frames=0

# free up memory
camera.release()
cv2.destroyAllWindows()


[STATUS] please wait! calibrating...
[STATUS] calibration successfull...
[STATUS] please wait! calibrating...
[STATUS] calibration successfull...
[STATUS] please wait! calibrating...
[STATUS] calibration successfull...
